In [1]:
# -*- encoding: utf-8 -*-
from __future__ import print_function  
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [2]:
from gensim import models
import logging

In [3]:
from keras.preprocessing.text import Tokenizer  
from keras.preprocessing.sequence import pad_sequences  
from keras.utils.np_utils import to_categorical  
from keras.layers import Dense, Input, Flatten,Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding  
from keras.models import Model  
from keras.optimizers import *  
from keras.models import Sequential  
from keras.layers import Merge 

In [4]:
import numpy as np  
import pandas as pd

In [5]:
from sklearn.cross_validation import train_test_split

In [6]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [11]:
def trainModel(data,target,modelFile):
    # 划分数据集
    train_set,test_set,train_tag,test_tag = train_test_split(data,target, test_size = 0.3)
    
    # 把一维的标签做onehot，pd.get_dummies的结果是df,把df转为ndarray (as_matrix())
    train_label = pd.get_dummies(train_tag).as_matrix()
    test_label = pd.get_dummies(test_tag).as_matrix()
    
    print('shape of train set:',train_set.shape)
    print('shape of train label:',train_label.shape)
    print('shape of test set:',test_set.shape)
    print('shape of test label:',test_label.shape)
    
    # 读入模型   
    model = models.Word2Vec.load(modelFile)
    # 读入word2vec模型提供的嵌入层，权重需要训练
    model_embedding = model.wv.get_embedding_layer(train_embeddings=False)

    # 训练嵌入层权重
    kmodel = Sequential()
    kmodel.add(model_embedding)
    kmodel.compile('rmsprop', 'mse')
    
#     print(model_embedding.get_weights())
        
    # 第一个卷积（卷积窗口是5*200）
    model = Sequential() 

    # 定义嵌入层
    # trainable=True 通过训练来更新权重
    # trainable=Fasle 由于使用了word2vec提供的权重，这里不用再训练了
    embedding_layer = Embedding(input_dim = model_embedding.input_dim,
                                output_dim = 50,
                                input_length = 200,
                                weights = model_embedding.get_weights(),
                                trainable = False)
    # 使用word2vec训练好的嵌入层
    model.add(embedding_layer)
    
    model.add(Conv1D(50, 3, padding='same',activation='relu')) 
    model.add(MaxPooling1D(2)) 
    model.add(Dropout(0.25))
    
    model.add(Conv1D(100, 3, padding='same',activation='relu')) 
    model.add(MaxPooling1D(2)) 
    model.add(Dropout(0.25))
    
    model.add(Flatten())  
    
    model.add(Dense(50, activation='relu')) # 全连接层  
    model.add(Dense(20, activation='relu')) # 全连接层  
    model.add(Dense(10, activation='softmax')) # softmax，输出文本属于20种类别中每个类别的概率  
        
    # 优化器我这里用了adadelta，也可以使用其他方法  
    model.compile(loss='categorical_crossentropy',  
                  optimizer='Adadelta',  
                  metrics=['accuracy'])  

    # =下面开始训练，nb_epoch是迭代次数，可以高一些，训练效果会更好，但是训练会变慢  
    model.fit(train_set, train_label,nb_epoch=10,batch_size = 100)  

    score = model.evaluate(train_set, train_label, verbose=0) # 评估模型在训练集中的效果，准确率约99%  
    print('train score:', score[0])  
    print('train accuracy:', score[1])  
    
    score = model.evaluate(test_set,test_label, verbose=0)  # 评估模型在测试集中的效果，准确率约为97%，迭代次数多了，会进一步提升  
    print('Test score:', score[0])  
    print('Test accuracy:', score[1]) 

In [8]:
def main():
    
    inFile = "./data/train_cnn.npy"
    modelFile = "./model/w2v_dim50.mdl"
    targetData = "./data/target.npy"
    vecFile = "./model/vector.bin"
    
    data_all = np.load(inFile)
    target = np.load(targetData)
    
#     print(data_all[:10,])
#     print(target[:10])
    # 训练模 
    trainModel(data_all,target,modelFile)  
    
    # 测试模型
#     useModel(modelFile)

In [12]:
if __name__ == '__main__':
    main()